In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
from keras.layers import Conv2D, MaxPooling2D, Input, Activation, Conv2DTranspose, concatenate, BatchNormalization, Add, Dropout
from keras.models import Model,load_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD
from keras import backend as K

import tensorflow as tf
import numpy as np

In [0]:
x_train = np.load('/content/drive/My Drive/Kaggle/TGS-Salt-Segmentation/data/101/all_samples/x_train.npy')
x_valid = np.load('/content/drive/My Drive/Kaggle/TGS-Salt-Segmentation/data/101/all_samples/x_valid.npy')
y_train = np.load('/content/drive/My Drive/Kaggle/TGS-Salt-Segmentation/data/101/all_samples/y_train.npy')
y_valid = np.load('/content/drive/My Drive/Kaggle/TGS-Salt-Segmentation/data/101/all_samples/y_valid.npy')


In [0]:
def get_iou_vector(A, B):
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch]>0, B[batch]>0
        
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        iou = (np.sum(intersection > 0) + 1e-10 )/ (np.sum(union > 0) + 1e-10)
        thresholds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in thresholds:
            s.append(iou > thresh)
        metric.append(np.mean(s))
        
    return np.mean(metric)
        
def iou2(label, pred):
    return tf.py_func(get_iou_vector, [label, pred > 0], tf.float64)
  
  
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------

def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

def lovasz_loss(y_true, y_pred):
    y_true, y_pred = K.cast(K.squeeze(y_true, -1), 'int32'), K.cast(K.squeeze(y_pred, -1), 'float32')
    #logits = K.log(y_pred / (1. - y_pred))
    logits = y_pred #Jiaxin
    loss = lovasz_hinge(logits, y_true, per_image = True, ignore = None)
    return loss

In [0]:
def batch_activation(x):
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  return x
  
def conv2d_block(input_tensor, n_filters, kernel_size=(3,3), activation=True):
    # first layer
    c1 = Conv2D(filters=n_filters, kernel_size=kernel_size, kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if activation:
      c1 = batch_activation(c1)
    return c1

  
def residual_block(input_tensor, num_filters=16, batch_activate = False):
    x = batch_activation(input_tensor)
    x = conv2d_block(x, num_filters, (3,3))
    x = conv2d_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, input_tensor])
    if batch_activate:
        x = batch_activation(x)
    return x

In [0]:
def unet_resnet_model():
    inputs = Input((101,101,1))

    #   101 -> 50
    filters = 16
    c1 = Conv2D(filters=filters, kernel_size=(3, 3), activation=None, padding="same")(inputs)
    print(type(c1))
    c1 = residual_block(c1, filters)
    c1 = residual_block(c1, filters, batch_activate=True)
    p1 = MaxPooling2D((2,2))(c1)

    #   50 -> 25
    filters = 32
    c2 = Conv2D(filters, (3, 3), activation=None, padding="same")(p1)
    c2 = residual_block(c2, filters)
    c2 = residual_block(c2, filters, batch_activate=True)
    p2 = MaxPooling2D((2,2))(c2)

    #   25 -> 12
    filters = 64
    c3 = Conv2D(filters, (3, 3), activation=None, padding="same")(p2)
    c3 = residual_block(c3, filters)
    c3 = residual_block(c3, filters, batch_activate=True)
    p3 = MaxPooling2D((2,2))(c3)

    #   12 -> 6
    filters = 128
    c4 = Conv2D(filters, (3, 3), activation=None, padding="same")(p3)
    c4 = residual_block(c4, filters)
    c4 = residual_block(c4, filters, batch_activate=True)
    p4 = MaxPooling2D((2,2))(c4)

    #  middle 6 -> 6
    filters = 256
    cm = Conv2D(filters, (3,3), activation=None, padding='same')(p4)
    cm = residual_block(cm, filters)
    cm = residual_block(cm, filters)

    #   upsampling
    # 6 -> 12
    filters = 128
    u4 = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding="same")(cm)
    u4 = concatenate([u4, c4])

    u4 = Conv2D(filters, (3, 3), activation=None, padding="same")(u4)
    u4 = residual_block(u4, filters)
    u4 = residual_block(u4, filters, batch_activate=True)

    # 12 -> 25
    filters = 64
    u3 = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding="valid")(c4)
    u3 = concatenate([u3, c3])

    u3 = Conv2D(filters, (3, 3), activation=None, padding="same")(u3)
    u3 = residual_block(u3, filters)
    u3 = residual_block(u3, filters, batch_activate=True)

    # 25 -> 50
    filters = 32
    u2 = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding="same")(c3)
    u2 = concatenate([u2, c2])

    u2= Conv2D(filters, (3, 3), activation=None, padding="same")(u2)
    u2 = residual_block(u2, filters)
    u2 = residual_block(u2, filters, batch_activate=True)

    # 50 -> 101
    filters = 16
    u1 = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding="valid")(c2)
    u1 = concatenate([u1, c1])

    u1= Conv2D(filters, (3, 3), activation=None, padding="same")(u1)
    u1 = residual_block(u1, filters)
    u1 = residual_block(u1, filters, batch_activate=True)

    output_layer = Conv2D(1, (1,1), padding="same")(u1)

    model = Model(inputs, output_layer)
    return model

  

In [0]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [0]:
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    
    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    
    # 12 -> 25
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)

    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    
    # 50 -> 101
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv1)
    
    
    return output_layer

In [0]:
epochs = 100
init_lr = 1e-2
K.clear_session()
sgd = SGD(lr=init_lr, momentum=0.9, decay=init_lr / epochs)
adam = Adam(lr=init_lr)
monitor = 'val_iou2'
monitor_mode = 'max'
model_callbacks = [
    EarlyStopping(monitor=monitor, mode=monitor_mode, patience=12, verbose=1),
    ReduceLROnPlateau(monitor=monitor, mode=monitor_mode, factor=0.1, patience=3, min_lr=1e-9, verbose=1)
]

# model= unet_resnet_model()
input_layer = Input((101,101,1))
model = Model(input_layer, build_model(input_layer=input_layer, start_neurons=16))
model.compile(optimizer=adam, loss=lovasz_loss, metrics=[iou2])
print(len(model.layers))
model.summary()

In [0]:
model.save('/content/drive/My Drive/Kaggle/TGS-Salt-Segmentation/models/unet_resnet_101_adam_lovasz-loss_iou-metric.h5')

In [33]:
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_valid, y_valid), callbacks=model_callbacks)

Train on 3200 samples, validate on 800 samples
Epoch 1/100
3200/3200 [==============================] - 65s 20ms/step - loss: 1.0864 - iou2: 0.2249 - val_loss: 1.1250 - val_iou2: 0.3500
Epoch 2/100
3200/3200 [==============================] - 53s 17ms/step - loss: 1.0073 - iou2: 0.2813 - val_loss: 1.0049 - val_iou2: 0.3900
Epoch 3/100
1472/3200 [============>.................] - ETA: 26s - loss: 1.0064 - iou2: 0.2878

KeyboardInterrupt: ignored

In [0]:
!cd '/content/drive/My Drive/TGS-Salt-Segmentation'

In [0]:
!git status

fatal: not a git repository (or any of the parent directories): .git
